# Weather Data
### by Lin van Nefterik and Stoil Iliev

Packages:

In [ ]:
!pip install numpy pandas matplotlib seaborn sklearn scipy

Imports:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class WeatherGraphProcessor:
  def __init__(self, stations_file="weather_stations.csv", data_file="weather_data.csv"):
    self.stations_df = None
    self.weather_df = None
    self.load_data(stations_file, data_file)
    self.GWS = None # Weather station graph
    self.eigenvalues = None
    self.eigenvects = None

  def load_data(self, stations_file, data_file):
    try:
      # Load weather station data
      self.stations_df = pd.read_csv(stations_file)
      print.(f"Loaded {len(self.stations_df)} weather stations")

      # Load weather measurement data
      self.weather_df = pd.read_csv(data_file)
      print.(f"Loaded {len(self.weather_df)} weather measurements")

      # Display basic info
      print("\nWeather stations columns:", list(self.stations_df.columns))
      print("Weather data columns:", list(self.weather_df.columns))
      print(f"Date range: {self.weather_df['DATE'].min()} to {self.weather_df['DATE'].max()}")

    except FileNotFoundError:
      print("File not found")

In [ ]:
def construct_weather_station_graph(self, alpha=0.1, beta=0.001, sparsification='knn', k=8, threshold=0.1):
  # alpha: distance decay
  # beta: alt diff decay
  # sparsification: knn, threshold, none?
  # k: kNN number
  # threshold: for weight-based sparisification

  n_stations = len(self.stations_df)
  W = np.zeros((n_stations, n_stations))

  # get coordinates / altitudes
  lats = self.stations_df['LAT'].values
  longs = self.stations_df['LONGI'].values
  heis = self.stations_df['HEI'].values

  # compute pairwise distances

  # sparsification

  # compute eigenvals, eigenvects

  # ...

In [ ]:
def construct_spacetime_graph(self, time_steps):

In [ ]:
def extract_signal(self, start_date, end_date):

In [ ]:
def graph_fourier_transform(self, signal):

In [ ]:
def inverse_graph_fourier_transform(self, gft_coeff):